In [ ]:
# Loading Required Libraries
import numpy as np
import json
import requests
from PIL import Image
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from matplotlib import style
import os
import glob
import random
from tensorflow.keras.preprocessing.image import img_to_array
from flask import Flask
from flasgger import Swagger

results=[]

def infer():
    # Enter Value between 1 and 8
    choose_index=3

    # load image files from the dataset
    image_files = [f for f in glob.glob(os.getcwd() + "/sample_images/**", recursive=True) if not os.path.isdir(f)]
    random.shuffle(image_files)

    # Creating Data and labels
    data = []
    labels = []
    img_dims = (96,96,3)
    pred_view=[]

    for img in image_files[:choose_index]:
        image = cv2.imread(img)
        
        try:
            image = cv2.resize(image,(img_dims[0],img_dims[1]))
        except:
            continue
            
        image = img_to_array(image)
        data.append(image)
        X=[]
        X.append(image)

        label = img.split(os.path.sep)[-1] # 'C:\\Users\\anura\\Desktop\\UTKface\\Data\\part1\\14_1_0_20170109204358190.jpg'
        if label[label.index('_')+1] == "1": # Female
            label = 1
        else:
            label = 0 # Male
            
        labels.append([label]) # [[1], [0], [0], ...]
        
        X=np.array(X, dtype="float") / 255.0
        
        input_data={"signature_name": "serving_default","instances": X.tolist()}
        payload =json.dumps(input_data)
        headers = {"content-type":"application/json"}
        url= "http://127.0.0.1:8501/v1/models/gender_model:predict"
        response = requests.post(url,data = payload, headers = headers)
        predictions =json.loads(response.text)['predictions']
        pred_view.append(predictions)
        prediction = np.argmax(predictions)
        results.append(prediction)
        

        with open('raw.txt', 'w') as raw_file:
            raw_file.write(json.dumps(input_data))

        # Visualizing Results
        plt.figure(figsize=(5,5))
        actual = label
        if prediction!=actual:
            col='r'
        else:
            col='g'

        image = plt.imread(img)
        plt.imshow(image)
        plt.xlabel('Actual={} || Pred={}'.format(actual,prediction),color=col,fontsize=20)
        plt.show()

    # pre-processing - conberting the image data to array
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)

infer()